In [12]:
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import time
import math

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [14]:
from tensorflow.examples.tutorials.mnist import input_data

In [15]:
mnist = input_data.read_data_sets("data/", one_hot = True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [16]:
batch_size = 128
n_classes = 10

In [17]:
keep_rate = 0.8
keep_prob = tf.placeholder(tf.float32)

In [18]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, n_classes])

In [19]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')
def maxpool2d(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [20]:
def convolutional_neural_network(x):
    weights = {'W_conv1': tf.Variable(tf.random_normal([3,3,1,32])),
                'W_conv2': tf.Variable(tf.random_normal([3,3,32,64])),
               'W_fc': tf.Variable(tf.random_normal([7*7*64, 1024])),
               'out': tf.Variable(tf.random_normal([1024, n_classes]))
              }
    
    biases = {'b_conv1': tf.Variable(tf.random_normal([32])),
              'b_conv2': tf.Variable(tf.random_normal([64])),
              'b_fc': tf.Variable(tf.random_normal([1024])),
              'out': tf.Variable(tf.random_normal([n_classes])),
    }
    
    x = tf.reshape(x, shape=[-1,28,28,1])
    conv1 = conv2d(x, weights['W_conv1'])+biases['b_conv1']
    conv1 = tf.nn.relu(conv1)
    conv1 = maxpool2d(conv1)
    
    conv2 = conv2d(conv1, weights['W_conv2'])+biases['b_conv2']
    conv2 = tf.nn.relu(conv2)
    conv2 = maxpool2d(conv2)
    
    fc = tf.reshape(conv2, [-1, 7*7*64])
    fc = tf.matmul(fc, weights['W_fc']) + biases['b_fc']
    fc = tf.nn.relu(fc)
    fc = tf.nn.dropout(fc, keep_rate)
    
    output = tf.matmul(fc, weights['out']) + biases['out']
    
    return output

In [21]:
def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 20
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss+=c
            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
            correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

In [22]:
train_neural_network(x)

Epoch 0 completed out of 20 loss: 945580.1108398438
Epoch 1 completed out of 20 loss: 123756.98294448853
Epoch 2 completed out of 20 loss: 64135.60024833679
Epoch 3 completed out of 20 loss: 40939.48117160797
Epoch 4 completed out of 20 loss: 29976.338383436203
Epoch 5 completed out of 20 loss: 21033.47698456049
Epoch 6 completed out of 20 loss: 15923.538763523102
Epoch 7 completed out of 20 loss: 12782.876502290368
Epoch 8 completed out of 20 loss: 10468.096100091934
Epoch 9 completed out of 20 loss: 8775.063146054745
Epoch 10 completed out of 20 loss: 7060.267597378232
Epoch 11 completed out of 20 loss: 6943.423261245713
Epoch 12 completed out of 20 loss: 5535.895692221912
Epoch 13 completed out of 20 loss: 4767.758868733231
Epoch 14 completed out of 20 loss: 3742.1950866300613
Epoch 15 completed out of 20 loss: 3770.960674008527
Epoch 16 completed out of 20 loss: 3501.419265395777
Epoch 17 completed out of 20 loss: 2990.607310793712
Epoch 18 completed out of 20 loss: 2405.5044066673